In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess, Fourier

In [ ]:
path = '../input/store-sales-time-series-forecasting/'

In [ ]:
# Tablica ze wszystkimi datami 
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

# Czytanie danych z pliku oil.csv
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')

# scalenie kalendarza z danymi z pliku oil.csv w celu uzupelnienia brakujacych dat
calendar = calendar.merge(data_oil, how='left', left_index=True, right_index=True)
# uzupelnienie brakujacych wartości metodą backfill
calendar['dcoilwtico'].fillna(method='backfill', inplace=True)

calendar

In [ ]:
# wykres ceny ropy
import seaborn as sns
sns.set_style('darkgrid')

_ = sns.lineplot(data = data_oil.dcoilwtico)

In [ ]:
# Czytamy Eventy gdzie jest swieto
df_hev = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True)

# Podmiana indeksu + sortowanie danych po dacie
df_hev = df_hev.set_index('date').sort_index()

# Pozostawiamy jedynie swieta narodowe dla ulatwienia 
df_hev = df_hev[df_hev.locale == 'National'] 

# Pozostawiamy 1 event dla kazdego dnia
df_hev = df_hev.groupby(df_hev.index).first() 

In [ ]:
# dołaczenie danych z events do oil

# dodatnie dni tygdnia
calendar['dofw'] = calendar.index.dayofweek
calendar['wd'] = True
calendar.loc[calendar.dofw > 4, 'wd'] = False

# polaczenie eventów do struktury calendar
calendar = calendar.merge(df_hev, how='left', left_index=True, right_index=True)

# jesli typ swieta = "bridge" to uznajemy dzien za wolny 
calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False

# jesli typ swieta = "work day" to uznajemy dzien za roboczy 
calendar.loc[calendar.type == 'Work Day', 'wd'] = True

# jesli typ swieta = "transfer" to uznajemy dzien za wolny 
calendar.loc[calendar.type == 'Transfer', 'wd'] = False

# jesli typ swieta = "holiday" oraz swieto nie zostalo przeniesione to uznajemy dzien za wolny 
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False

# jesli typ swieta = "holiday" oraz swieto zostalo przeniesione to uznajemy dzien za roboczy 
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True

# usunięcie niepotrzebnych kolumn
calendar.drop(['locale', 'locale_name', 'description', 'transferred'], axis = 1, inplace = True) 

calendar

In [ ]:
# rozbicie dofw na wartości 0/1 na 7 kolumn
calendar = pd.get_dummies(calendar, columns = ['dofw']) 

# rozbicie type na wartości 0/1
calendar = pd.get_dummies(calendar, columns = ['type']) 

calendar = calendar.to_period('D')

calendar

In [ ]:
# Czytamy dane treningowe

df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv',
                    parse_dates = ['date'], infer_datetime_format = True,
                    dtype = {'store_nbr' : 'category', 'family' : 'category'},
                    usecols = ['date', 'store_nbr', 'family', 'sales'])

# stworzenie kolumn do wykresów
df_train['day_of_week'] = df_train['date'].dt.day_name()
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day

df_train['date'] = df_train.date.dt.to_period('D')

df_train = df_train.set_index(['date', 'store_nbr', 'family']).sort_index()
df_train

In [ ]:
# Wykres sprzedarzy w danym miesiącu

import plotly.express as px
by_month_df = df_train.groupby(['month'])['sales'].mean().reset_index()
fig = px.bar(by_month_df, x='month', y='sales', color='sales', color_continuous_scale="sunset")
fig.show()


In [ ]:
# Wykres sprzedaży danego dnia miesiąca

import plotly.graph_objs as go
by_day_df = df_train.groupby(['day'])['sales'].mean().reset_index()
fig = go.Figure(data=go.Scatter(x=by_day_df['day'], 
                                y=by_day_df['sales'],
                                marker_color='blue', text="sales"))
fig.show()

In [ ]:
# wykres sprzedaży danego dnia tygodnia 

by_weekday_df = df_train.groupby(['day_of_week'])['sales'].mean()
new_order_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
by_weekday_df = by_weekday_df.reindex(new_order_week, axis=0).reset_index()
fig = px.bar(by_weekday_df, x='day_of_week', y='sales', color='sales', color_continuous_scale="sunset")
fig.show()

In [ ]:
# usunięcie zbędynych kolumn

df_train.drop(columns = ['month', 'day', 'day_of_week', 'year'], inplace = True)

In [ ]:
# ustawienie daty poczatkowej i koncowej dla danych treningiowych

sdate = '2017-04-30' 
edate = '2017-08-15'

# ograniczenie danych treningowych
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]

In [ ]:
# wyświetlenie wykresów dla poszczególnych rodzin produktów
family = {c[2] for c in df_train.index}
for f in family :
    ax = y.loc(axis = 1)['sales', :, f].plot(legend = None)
    ax.set_title(f)

In [ ]:
# Utworzenie szeregów kalendarza furiera
fourier = CalendarFourier(freq = 'W', order = 4)

# utworzenie procesu deterministycznego
dp = DeterministicProcess(index = y.index,
                          order = 1,
                          seasonal = False,
                          constant = False,
                          additional_terms = [fourier],
                          drop = True)

#utworzenie zbioru treningowego 
X = dp.in_sample()

#połączenie df train i calendar
X = X.join(calendar)

X

In [ ]:
# utworzenie df testowego
numOfPredictedDays = 16

X_test = dp.out_of_sample(steps = numOfPredictedDays) 

# dołączenie calendar do danych testowych
X_test = X_test.join(calendar)

X_test

In [ ]:
# utworzenie modelu regresyjnego
model = Ridge(fit_intercept=True, solver='auto', alpha=0.4, normalize=True)
model.fit(X, y)

# utworzenie tablicy predykcji
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [ ]:
# wyznaczenie predykcji zbioru treningowego
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

# sprzedaż powinna byc wieksza niż 0
y_target['sales_pred'] = y_pred['sales'].clip(0.)

# obliczenie mean_squared_log_error
pred1 = y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred']))

display(y_target)
display(pred1)

In [ ]:
# utworzenie predykcji dla zbioru testowego  
sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])

# sprzedaż powinna byc wieksza niż 0
sales_pred[sales_pred < 0] = 0. 

display(sales_pred)

In [ ]:
# Utworzenie pliku submission

df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred.values
df_sub.to_csv('submission.csv', index=True)